In [28]:
import keen
from keen.client import KeenClient
import arrow

KEEN_PROJECT_ID = "572dfdae3831443195b2f30c"
KEEN_READ_KEY = "5de7f166da2e36f6c8617347a7a729cfda6d5413db8d88d7f696b61ddaa4fe1e5cdb7d019de9bb0ac846d91e83cdac01e973585d0fba43fadf92f06a695558b890665da824a0cf6a946ac09f5746c9102d228a1165323fdd0c52c92b80e78eca"

client = KeenClient(
    project_id=KEEN_PROJECT_ID,
    read_key=KEEN_READ_KEY
)

# https://earthquake.usgs.gov/data/comcat/data-eventterms.php
earthquake_extraction = client.extraction("earthquakes", timeframe={
    "start": arrow.get(2016, 10, 1).format(),
    "end": arrow.get(2017, 2, 28).format()
})

[A while ago, we created a basic example](https://blog.keen.io/analyzing-your-keen-io-data-with-jupyter-notebook-aefe0672da25) that analyzed earthquakes using
the [Keen IO Python Client]() with [Jupyter Notebook]().
This week we're going
to be looking at creating quick visualizations in Python using a visualization library called Bokeh.

I mentioned in my last blog post that I wanted to talk about visualizing data
with Python using Matplotlib, but some of the developers of Jupyter notebook highly recommended [Bokeh](http://bokeh.pydata.org/en/latest/) instead.
Matplotlib can have a somewhat confusing interface,
where as Bokeh is a little easier to get up and running quickly.
Bokeh also lets users create interactive graphs in Jupyter Notebook,
letting users understand their data deeper than static graphs.

To install Bokeh, run `pip install bokeh`. After Bokeh is installed, start
Jupyter notebook by running `jupyter notebook`.

Let's begin by running the query similar to the one we used last time,
earthquake counts by day.

In [29]:
earthquakes_by_day = client.count_unique("earthquakes",
    timeframe={
        "start": arrow.get(2017, 6, 12).format(),
        "end": arrow.get(2017, 7, 12).format()
    },
    target_property="id",
    interval="daily"
)

Now we're ready to use Bokeh to visualize this data. Run the code below in a new cell.

In [30]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

# from bokeh.io import reset_output
# reset_output()

Loading BokehJS ...

The first line imports `figure` and `show`, two functions that will let us plot our data.
The next line imports a function called `output_notebook`.
We need to run this method before we start plotting data so our plots are drawn below our
notebook cells.

A line graph would be a great choice to plot this data. In order to plot this, we need to pull out the number of earthquakes for a timeframe and the corresponding date.

In [32]:
y = list(map(lambda row: row["value"], earthquakes_by_day))
x = list(map(lambda row: arrow.get(row["timeframe"]["start"]).datetime.replace(tzinfo=None), earthquakes_by_day))
pl = figure(title="Number of Earthquakes per Day", x_axis_type="datetime")
pl.line(x, y)


show(pl)

`y` is the numeric value, `x` is the datetime value
`figure` initializes the chart object. 
`line` takes lists of the x and y values and plots.

`show(pl)` is the line that draws the chart in our notebook.


Bokeh even lets us add tooltips to our charts! 
We can call `from bokeh.models import HoverTool` and pass a new instance of `HoverTool` to
our `figure` object.

In [44]:
from bokeh.models import HoverTool

# You can configure the hover object to customize the tooltips
# See http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#setting-tool-visuals

pl = figure(title="Number of Earthquakes per Day",
            x_axis_type="datetime")
pl.line(x, y)

hover = HoverTool(tooltips=[
        ("Date", "@x{%F}"),
        ("Count", "@y{int}")
    ],
    formatters={"x": "datetime"},
    mode='vline'
)

pl.add_tools(hover)

show(pl)

Notice that there are a lot of earthquakes on 6/17!
This might be an interesting place to dive deeper. Next time, we'll plot  the earthquakes that happened in that time period using [Basemap](https://matplotlib.org/basemap/) and see if we can find any interesting.